In [ ]:
# modified from Pytorch documentation: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [9]:
import pandas as pd
import numpy as np
import datetime


from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
usa_df = pd.read_csv(url)

In [3]:
usa_case_df = usa_df.filter(like='/20' ) # regex

In [172]:
usa_case_arr = usa_case_df.to_numpy()
case_max = np.max(usa_case_arr)
usa_case_arr = np.log10(1 + usa_case_arr)/np.log10(1 + case_max)
case_mean = np.mean(  usa_case_arr )
usa_case_arr = usa_case_arr - case_mean
np.max(usa_case_arr), np.min(usa_case_arr)

def transform(arr, arr_max, arr_mean, inverse=False):
    if inverse:
        ret = arr + arr_mean
        ret = ret * np.log10(1+arr_max)
        ret = 10**ret - 1
    else:
        tmp = np.log10(1+arr)/np.log10(1+arr_max)
        ret = tmp - arr_mean
    return ret
    
    

day_vec = np.array( [  datetime.datetime.strptime(d, '%m/%d/%y').weekday()  for d in usa_case_df.columns ] )

# Data

In [11]:
class covid_dataset(Dataset):
    def __init__(self,  arr):
        self.arr = arr
        no_region, no_day = arr.shape
        self.no_region = no_region
        self.no_day = no_day
    def __len__(self):
        return  self.no_region
    def __getitem__(self, idx):
        x = torch.tensor( self.arr[idx,:], dtype=torch.float)
        return x

np.random.seed(2020)
n_sample = usa_case_arr.shape[0]
idx = np.random.permutation( n_sample )
no_tr = round(n_sample * .6)
no_va = round(n_sample * 0.2 )
usa_case_arr_tr = usa_case_arr[ :no_tr , :]
usa_case_arr_va = usa_case_arr[no_tr: (no_tr + no_va), :]
usa_case_arr_te = usa_case_arr[(no_tr + no_va):, :]

batch_size = 128




def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    def __len__(self):
        return len(self.dl)


tr_ds = covid_dataset(usa_case_arr_tr)
va_ds = covid_dataset(usa_case_arr_va)
te_ds = covid_dataset(usa_case_arr_te)
tr_dl = DataLoader(tr_ds,  batch_size=batch_size, shuffle=True, drop_last=True)
va_dl = DataLoader(va_ds,  batch_size=len(va_ds), shuffle=False, drop_last=False)
te_dl = DataLoader(te_ds,  batch_size=len(te_ds), shuffle=False, drop_last=False)


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

tr_dl = DeviceDataLoader(tr_dl, device)
va_dl = DeviceDataLoader(va_dl, device)
te_dl = DeviceDataLoader(te_dl, device)




In [12]:
x  = next( iter(tr_dl) )

In [13]:
x.shape

torch.Size([128, 88])

# Model

In [131]:
input_dim = 1
seq_len = len(day_vec)
output_dim = 1
hidden_dim = 4
num_layers = 1
dropout = 0.5 if num_layers > 1 else 0
batch_size = 1
MAX_LENGTH = seq_len

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size)

    def forward(self, input, hidden):
        output =  input # 1, 1, -1
        output, hidden = self.gru(output, hidden) # (seq_len = 1,1, 1), (1, 1, hidden_size)
        return output, hidden #(seq_len, batch=1, input=1), (1, batch=1, hidden_size)

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(output_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden): #(1, 1,1), (1,1, hidden)
        output = input # 1,1,-1
        output = F.relu(output)
        output, hidden = self.gru(output, hidden) #  (seq_len = 1,1, 1), (1, 1, hidden_size) (seq_len, batch, input_size), (num_layers * num_directions, batch, hidden_size)
        output =  self.out(output)
        return output, hidden  #  (seq_len, batch, num_directions * hidden_size), (num_layers * num_directions, batch, hidden_size)

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden() # (1, 1, hidden_dim)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei].unsqueeze(2), encoder_hidden) # input_tensor[ei] = 1, 1, 
        encoder_outputs[ei] = encoder_output[0, 0]  # 1 dim hidden size


    decoder_hidden = encoder_hidden # (1, 1, hidden)
    decoder_input = torch.tensor([[0.0]], device=device).view(1,1,1)  # (1, 1)

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di].unsqueeze(2))
            decoder_input = target_tensor[di].unsqueeze(2) # feed reversel


    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            decoder_input = decoder_output.detach()  # detach from history as input
            loss += criterion(decoder_output, target_tensor[di].unsqueeze(2)).pow(2)
#             print('decoder')
#             print(decoder_output)
#             print('target')
#             print(target_tensor[di])


    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item()/input_length

In [132]:
import random
random.choice(usa_case_arr_tr).shape
usa_case_arr_tr.shape

(1953, 88)

In [137]:

import time
import random

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


def trainIters(encoder, decoder, n_iters, print_every=1000,  learning_rate=1e-5):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_tensors = [torch.tensor( random.choice(usa_case_arr_tr), dtype=torch.float).view(-1, 1, 1)
                      for i in range(n_iters)]
    criterion = nn.MSELoss()

    for iter in range(1, n_iters + 1):
        input_tensor = training_tensors[iter - 1] #   seq_len, 1, 1
#         target_tensor =  input_tensor[range(-1, -len(input_tensor), -1)].clone()
        target_tensor = input_tensor.clone()

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
#         print(loss)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))


In [138]:
encoder = EncoderRNN(input_dim, hidden_dim)
decoder = DecoderRNN(hidden_dim, output_dim)

In [139]:
trainIters(encoder, decoder, 7500, 100)

0m 8s (- 10m 31s) (100 1%) 0.0149
0m 16s (- 10m 18s) (200 2%) 0.0129
0m 24s (- 9m 56s) (300 4%) 0.0084
0m 32s (- 9m 39s) (400 5%) 0.0099
0m 40s (- 9m 26s) (500 6%) 0.0121
0m 48s (- 9m 15s) (600 8%) 0.0075
0m 56s (- 9m 5s) (700 9%) 0.0091
1m 3s (- 8m 55s) (800 10%) 0.0091
1m 11s (- 8m 46s) (900 12%) 0.0079
1m 20s (- 8m 40s) (1000 13%) 0.0067
1m 28s (- 8m 34s) (1100 14%) 0.0067
1m 36s (- 8m 25s) (1200 16%) 0.0072
1m 44s (- 8m 18s) (1300 17%) 0.0070
1m 52s (- 8m 10s) (1400 18%) 0.0063
2m 0s (- 8m 2s) (1500 20%) 0.0057
2m 8s (- 7m 54s) (1600 21%) 0.0050
2m 17s (- 7m 47s) (1700 22%) 0.0107
2m 25s (- 7m 40s) (1800 24%) 0.0091
2m 33s (- 7m 31s) (1900 25%) 0.0086
2m 41s (- 7m 23s) (2000 26%) 0.0107
2m 49s (- 7m 15s) (2100 28%) 0.0067
2m 57s (- 7m 7s) (2200 29%) 0.0059
3m 5s (- 7m 0s) (2300 30%) 0.0058
3m 13s (- 6m 52s) (2400 32%) 0.0072
3m 21s (- 6m 43s) (2500 33%) 0.0073
3m 30s (- 6m 36s) (2600 34%) 0.0077
3m 39s (- 6m 29s) (2700 36%) 0.0064
3m 47s (- 6m 21s) (2800 37%) 0.0080
3m 55s (- 6m 13

In [201]:
def validate(input_tensor, target_tensor, encoder, decoder,   max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden() # (1, 1, hidden_dim)

    encoder.eval()
    decoder.eval()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei].unsqueeze(2), encoder_hidden) # input_tensor[ei] = 1, 1, 
        encoder_outputs[ei] = encoder_output[0, 0]  # 1 dim hidden size


    decoder_hidden = encoder_hidden # (1, 1, hidden)
    decoder_input = torch.tensor([[0.0]], device=device).view(1,1,1)  # (1, 1)

    criterion = nn.MSELoss()

    # Without teacher forcing: use its own predictions as the next input
    for di in range(max_length):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden)
        decoder_input = decoder_output.detach()  # detach from history as input
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(2)).pow(2)
#     print(decoder_output)
#     print(target_tensor[di-1])
#             print('decoder')
#             print(decoder_output)
#             print('target')
#             print(target_tensor[di])

    return loss.item()/input_length

In [202]:
def get_decoding_err(arr):
    va_loss = 0
    for i in range(arr.shape[0]):
        input_tensor = torch.tensor(  arr[i,], dtype=torch.float).view(-1, 1, 1)
        target_tensor = input_tensor.clone()
        va_loss += validate(input_tensor, target_tensor, encoder, decoder)
    va_loss = va_loss/arr.shape[0]
    return va_loss

va_loss = get_decoding_err(usa_case_arr_va)
te_loss = get_decoding_err(usa_case_arr_te)
print(f'va_loss = {va_loss}, te_loss={te_loss}')

va_loss = 0.0012461274160678222, te_loss=0.0010210112583835028
